# Text generating with RNN

In [3]:
import numpy as np
import tensorflow as tf
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [4]:
text = """
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다"""

## Preprocessing

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

### Generating word set

In [6]:
t = Tokenizer()
t.fit_on_texts([text])

In [7]:
t.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

### Size of word set

In [8]:
vocab_size = len(t.word_index) + 1
vocab_size

12

In [9]:
t.texts_to_sequences(['그의 말이 법이다'])

[[6, 1, 7]]

### Sample sequence for learning

In [10]:
sequences = []
for i in text.split('\n') :
    encoded = t.texts_to_sequences([i])[0]
    for j in range(1, len(encoded)) :
        sequence = encoded[: j + 1]
        sequences.append(sequence)
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [11]:
max_len = max(len(s) for s in sequences)
max_len

6

### Padding

- Padding all samples length as the longest sample length
- 'pre' | padding with 0 for all front factors

In [12]:
sequences = pad_sequences(sequences, maxlen = max_len, padding = 'pre')
sequences[:5]

array([[0, 0, 0, 0, 2, 3],
       [0, 0, 0, 2, 3, 1],
       [0, 0, 2, 3, 1, 4],
       [0, 2, 3, 1, 4, 5],
       [0, 0, 0, 0, 6, 1]], dtype=int32)

In [13]:
X = sequences[:, : -1]
y = sequences[:, -1]

In [14]:
Y = to_categorical(y)   # to_categorical(y, vocab_size)
Y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

## Processing

### Model defining

- Embedding
- SimpleRNN

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [16]:
model = Sequential([
    Embedding(vocab_size, 4, input_length = max_len -1),
    SimpleRNN(32),
    Dense(vocab_size, activation = 'softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5, 4)              48        
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                1184      
_________________________________________________________________
dense (Dense)                (None, 12)                396       
Total params: 1,628
Trainable params: 1,628
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [18]:
history = model.fit(X, Y, epochs = 200, verbose = 0)
history.history['accuracy'][-1]

0.9090909361839294

### Verifying model

In [19]:
from my_util import sentence_generation

In [20]:
sentence_generation(model, t, max_len, '경마장에', 3)

'경마장에 말이 말이 뛰고'

In [21]:
sentence_generation(model, t, max_len, '그의', 2)

'그의 말이 법이다'

In [22]:
sentence_generation(model, t, max_len, '가는', 5)

'가는 말이 고와야 오는 말이 곱다'

## Model changing

- Embedding vector number : 2, 4, 6
- RNN node number : 24, 32, 48

In [23]:
def execute_model(n_embed, n_rnn) :
    model = Sequential([
                        Embedding(vocab_size, n_embed, input_length = max_len -1),
                        SimpleRNN(n_rnn),
                        Dense(vocab_size, activation = 'softmax')
                        ])
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    history = model.fit(X, Y, epochs = 200, verbose = 0)
    print(f"Accuracy : {history.history['accuracy'][-1]:.4f}")
    print(sentence_generation(model, t, max_len, '경마장에', 3))
    print(sentence_generation(model, t, max_len, '그의', 2))
    print(sentence_generation(model, t, max_len, '가는', 5)) 

In [25]:
for n_embed in [2, 4, 6]:
    for n_rnn in [24, 36, 48]:
        print('======================================')
        print(f'n_embed = {n_embed}, n_rnn = {n_rnn}')
        execute_model(n_embed, n_rnn)
        print()

n_embed = 2, n_rnn = 24
Accuracy : 0.8182
경마장에 말이 말이 말이
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed = 2, n_rnn = 36
Accuracy : 0.8182
경마장에 말이 말이 뛰고
그의 말이 법이다
가는 말이 말이 오는 말이 곱다

n_embed = 2, n_rnn = 48
Accuracy : 0.9091
경마장에 말이 법이다 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed = 4, n_rnn = 24
Accuracy : 0.9091
경마장에 말이 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed = 4, n_rnn = 36
Accuracy : 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed = 4, n_rnn = 48
Accuracy : 0.9091
경마장에 말이 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed = 6, n_rnn = 24
Accuracy : 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed = 6, n_rnn = 36
Accuracy : 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed = 6, n_rnn = 48
Accuracy : 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

